<a href="https://colab.research.google.com/github/hackerpranavpandey/Machine-Learning-Course/blob/main/Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf=RandomForestRegressor()

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving tested.csv to tested (1).csv


In [ ]:
df=pd.read_csv("tested.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [ ]:
df['Age'].mean()

30.272590361445783

In [ ]:
df['Age'].value_counts()

Age
21.0    17
24.0    17
22.0    16
30.0    15
18.0    13
        ..
76.0     1
28.5     1
22.5     1
62.0     1
38.5     1
Name: count, Length: 79, dtype: int64

In [ ]:
# df['Age'].fillna(df['Age'].mean(),inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [ ]:
y=df['Survived']

In [ ]:
df.drop(columns=['Survived'],inplace=True)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df,y,test_size=0.2,random_state=100)

In [ ]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
282,1174,3,"Fleming, Miss. Honora",female,NaN,0,0,364859,7.75,NaN,Q
275,1167,2,"Bryhl, Miss. Dagmar Jenny Ingeborg",female,20.0,1,0,236853,26.00,NaN,S
219,1111,3,"Thomson, Mr. Alexander Morrison",male,NaN,0,0,32302,8.05,NaN,S
69,961,1,"Fortune, Mrs. Mark (Mary McDougald)",female,60.0,1,4,19950,263.00,C23 C25 C27,S
127,1019,3,"McCoy, Miss. Alicia",female,NaN,2,0,367226,23.25,NaN,Q


In [ ]:
X_train.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
X_test.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [ ]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
282,3,female,NaN,0,0,7.75,Q
275,2,female,20.0,1,0,26.00,S
219,3,male,NaN,0,0,8.05,S
69,1,female,60.0,1,4,263.00,S
127,3,female,NaN,2,0,23.25,Q


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
numerical_features=['Age','Fare']
numerical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_features={'Embarked','Sex'}
categorical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor=ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_features),
        ('cat',categorical_transformer,categorical_features)
    ]
)

In [ ]:
from sklearn.linear_model import LinearRegression
clf=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',LinearRegression())
])

In [ ]:
from sklearn import set_config
set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  {'Embarked', 'Sex'})])),
                ('classifier', LinearRegression())])

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid={
    'processor_num__imputer__strategy':['mean','median'],
    'processor_cat_imputer_strategy':['most_frequent','constant']
}
grid_search=GridSearchCV(clf,param_grid,cv=10)